# Data Reshaping and Joining

## Reshaping

The same data can be represented in many different ways. Imagine we have some dataset that records how far different students can throw a ball at three different time points. Two common ways that data can be organized in a `.csv` file are called *wide* and *long*. Look at the difference in organization in the two datasets below:

In [14]:
library(tidyr)
library(dplyr)
d = expand.grid(student = c('Amy', 'Jack', 'Ben'), timepoint = paste0('Time', 1:3))
d$distance = rnorm(nrow(d), 80, 10)
d = arrange(d, student)
d
spread(d, timepoint, distance)

student,timepoint,distance
<fct>,<fct>,<dbl>
Amy,Time1,74.29328
Amy,Time2,80.63865
Amy,Time3,90.91206
Jack,Time1,71.34743
Jack,Time2,74.17298
Jack,Time3,88.02991
Ben,Time1,74.69437
Ben,Time2,87.21723
Ben,Time3,71.75157


student,Time1,Time2,Time3
<fct>,<dbl>,<dbl>,<dbl>
Amy,74.29328,80.63865,90.91206
Jack,71.34743,74.17298,88.02991
Ben,74.69437,87.21723,71.75157


Many functions in R will expect your data to be in the first format (ie, long format). You will sometimes encounter data in the second format (ie, wide format) and you will need to translate it to long format in order to analyze it. More specifically, many functions in R will expect that your data is organized such that:

1. Every column is a variable.  
2. Every row is an observation.  
3. Every cell is a single value.

This definition is a bit abstract, and what counts as a "variable" or "observation" in a given situation can be tricky, but we'll look at examples later that will hopefully make this more clear.

Even though it's not recommended to use wide data for analysis in R, sometimes wide format is useful for presenting analysis summaries in a table, so you may want to convert from long to wide to present results in the form of a table. 

Reshaping data can get quite tricky. We'll go over a framework in R for reshaping that will add valuable tools to your analysis toolkit.

## tidyr

The library of code called `tidyr` contains many functions needed to reshape data. The image below nicely summarizes the four functions we'll be working with:

<img src='reshaping.png' height=75% width=75%>

* `spread(data, key, value)`  
    * Used for converting from long to wide data  
    * `data`: Name of data frame that you want to manipulate  
    * `key`: The column that stores as its values the *names* you want to use for the new wide columns  
    * `value`: The column that stores as its values the *values* you want to use for the new wide columns
    

* `gather(data, key, value, column_to_gather_1:column_to_gather_3)`  
    * Used for converting from wide to long data  
    * `data`: Name of data frame that you want to manipulate  
    * `key`: The name of the new column that will store the column *names* of the wide data  
    * `value`: The name of the new column that will store the column *values* of the wide data  
    * `column_to_gather_`: Names of all columns to convert from wide to long
    
    
* `separate(data, col, into = c('new_column_1', 'new_column_2' ... ), sep)`  
    * Used to split one column into several  
    * When do you want to use this?  
        * It's most handy after gathering columns that represent different types of data.  
    * `data`: Name of the data frame that you want to manipulate  
    * `col`: Name of the column that you want to divide into many columns  
    * `into`: Desired names of new columns  
    * `sep`: What character to split the values in the old column on to form new columns  


* `unite(data, col, column_to_unite_1:column_to_unite_3, sep)`  
    * Used to unite many columns into one
    * When do you want to use this?  
        * It's most handy before spreading columns when you want to spread across columns that represent many different types of data.  
    * `data`: Name of the data frame that you want to manipulate  
    * `col`: Desired name of the new column created after uniting  
    * `column_to_unite_`: Name of columns to unite into one column  
    * `sep`: Character to separate the values in the new column

### Examples

**Spreading to wide**

Imagine we have some data in long format and we want to spread to wide. 

***How do I know if my data is in wide or long format?*** A key giveaway that our data is in long format is if an identifier for the observation (eg, participant name or number) is repeated within a column. Take the following example from before:

In [2]:
d

student,timepoint,distance
<fct>,<fct>,<dbl>
Amy,Time1,100.23316
Amy,Time2,90.10438
Amy,Time3,86.36776
Jack,Time1,58.96709
Jack,Time2,83.49879
Jack,Time3,69.53046
Ben,Time1,63.68355
Ben,Time2,90.18758
Ben,Time3,95.53796


Here we see that each student is represented on several rows, so it's a good indication that our data is in long format. Wide format, in this example, would be if each student's data occupied only a single row. 

***What do I want the new data to look like?*** Before trying to reshape data, it's always a good exercise to ask yourself: what is it that you want the new data to look like? What variables should be columns, what information should be represented as rows?

So here, we want one row per student (ie, three rows), we want to use the values in the `timepoint` variable as our new column names, and we want to use the values in the `distance` column to be the new values in the spread out columns. 

To do this, we use the `spread` function:

In [3]:
wide_data = spread(d, timepoint, distance)

wide_data

student,Time1,Time2,Time3
<fct>,<dbl>,<dbl>,<dbl>
Amy,100.23316,90.10438,86.36776
Jack,58.96709,83.49879,69.53046
Ben,63.68355,90.18758,95.53796


**Gathering to long**

Let's say we want to do some analyses on the wide data frame above. We'll need to convert it back to long format. To do this, we need to really visualize what we want that new data frame to look like. I see that there's one variable spread out across all columns two through four (ie, time) and each of the values in these columns represent on variable (ie, distance). So, in my `gather` function, I need to specify these two new columns (time and distance) as well as the existing columns that I want to gather:

In [4]:
long_data = gather(wide_data, time, distance, Time1:Time3)
long_data = arrange(long_data, student) ## sorting the data by a particular variable
long_data

student,time,distance
<fct>,<chr>,<dbl>
Amy,Time1,100.23316
Amy,Time2,90.10438
Amy,Time3,86.36776
Jack,Time1,58.96709
Jack,Time2,83.49879
Jack,Time3,69.53046
Ben,Time1,63.68355
Ben,Time2,90.18758
Ben,Time3,95.53796


**Uniting and spreading to wide**

What we've seen above so far are the simplest examples, where we're really only dealing with spreading or gathering two variables. But this can get more complicated. For example imagine we have another independent measure (ie, thing that we manipulated in our experiment) in the data:

In [5]:
d <- expand.grid(student = c('Amy', 'Jack', 'Ben'), time = paste0('Time', 1:3), weather = c('Sunny', 'Rainy'))
d$distance <- rnorm(nrow(d), 80, 10)
d = arrange(d, student)
d

student,time,weather,distance
<fct>,<fct>,<fct>,<dbl>
Amy,Time1,Sunny,67.26723
Amy,Time2,Sunny,100.47912
Amy,Time3,Sunny,79.53061
Amy,Time1,Rainy,89.15834
Amy,Time2,Rainy,78.03313
Amy,Time3,Rainy,67.87005
Jack,Time1,Sunny,69.06735
Jack,Time2,Sunny,82.19153
Jack,Time3,Sunny,92.37277


For each student, we now have three time points, and two types of weather per time point. We can spread the variables `time` and `weather` out as columns and fill in the values with `distance`.  

***How do I know if a certain variable can be spread in the column names from long to wide?*** If all participants saw *all* levels of that variable, then that variable can be spread to wide. For example, because all participants experienced both rainy and sunny weather across all time points, we can spread along that variable.

Besides the input data, spreading only takes two arguments: column to use as column names, and column to use as values. So we need to condense the data above to get it to that format first before we can spread it. `time` and `weather` are both the columns we want to use as column names, so we need to put those together (ie, unite!).

In [6]:
d1 <- unite(d, new_col_names, time:weather, sep = '_')
d1

student,new_col_names,distance
<fct>,<chr>,<dbl>
Amy,Time1_Sunny,67.26723
Amy,Time2_Sunny,100.47912
Amy,Time3_Sunny,79.53061
Amy,Time1_Rainy,89.15834
Amy,Time2_Rainy,78.03313
Amy,Time3_Rainy,67.87005
Jack,Time1_Sunny,69.06735
Jack,Time2_Sunny,82.19153
Jack,Time3_Sunny,92.37277


This is like an intermediary data frame, we're inbetween long and preparing to go to wide here. We joined `time` and `weather` together and separated them with a `_` in order to keep things organized. Now we just spread like we did before:

In [7]:
long_data = spread(d1, new_col_names, distance)
long_data

student,Time1_Rainy,Time1_Sunny,Time2_Rainy,Time2_Sunny,Time3_Rainy,Time3_Sunny
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Amy,89.15834,67.26723,78.03313,100.47912,67.87005,79.53061
Jack,64.27249,69.06735,90.20537,82.19153,80.42154,92.37277
Ben,92.98254,77.79877,69.65149,106.46793,79.15999,87.71887


And we've achieved our final wide shape: each participant has only one row, and all those variables that each participant saw every level of are represented as different columns.

***What if there's a variable in the data that each participant did not see each level of?*** We just leave that one out of the uniting and spreading function calls. I'll do the same operations but add in what's called a *between* subjects variable.

In [8]:
d$training_condition <- ifelse(d$student=='Amy' | d$student=='Jack', 'trained', 'control')
d1 = d[,c(1,5,2:4)]
d1

student,training_condition,time,weather,distance
<fct>,<chr>,<fct>,<fct>,<dbl>
Amy,trained,Time1,Sunny,67.26723
Amy,trained,Time2,Sunny,100.47912
Amy,trained,Time3,Sunny,79.53061
Amy,trained,Time1,Rainy,89.15834
Amy,trained,Time2,Rainy,78.03313
Amy,trained,Time3,Rainy,67.87005
Jack,trained,Time1,Sunny,69.06735
Jack,trained,Time2,Sunny,82.19153
Jack,trained,Time3,Sunny,92.37277


In [9]:
d2 = unite(d1, new_col_names, time:weather)
wide_data = spread(d2, new_col_names, distance)
wide_data

student,training_condition,Time1_Rainy,Time1_Sunny,Time2_Rainy,Time2_Sunny,Time3_Rainy,Time3_Sunny
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Amy,trained,89.15834,67.26723,78.03313,100.47912,67.87005,79.53061
Jack,trained,64.27249,69.06735,90.20537,82.19153,80.42154,92.37277
Ben,control,92.98254,77.79877,69.65149,106.46793,79.15999,87.71887


We still satisfy the requirement that each student have only one row, even though we didn't spread the `training_condition` variable.

**Gathering to long and separating**

Let's go from the data frame above back to a long data frame. We just invert the operations that we just did. So first we `gather`:

In [10]:
d1 = gather(wide_data, gathered_names, distance, Time1_Rainy:Time3_Sunny)
arrange(d1, student)


student,training_condition,gathered_names,distance
<fct>,<chr>,<chr>,<dbl>
Amy,trained,Time1_Rainy,89.15834
Amy,trained,Time1_Sunny,67.26723
Amy,trained,Time2_Rainy,78.03313
Amy,trained,Time2_Sunny,100.47912
Amy,trained,Time3_Rainy,67.87005
Amy,trained,Time3_Sunny,79.53061
Jack,trained,Time1_Rainy,64.27249
Jack,trained,Time1_Sunny,69.06735
Jack,trained,Time2_Rainy,90.20537


We're almost already there. But remember our three rules from the beginning of the notebook. Rule 1 says every column must be *one* variable. Right now, `gathered_names` is representing two variables. To correct for this, we spread it into two and split the values on the `_` character:

In [11]:
long_data = separate(d1, gathered_names, into = c('time', 'weather'), sep = '_')
arrange(long_data, student)

student,training_condition,time,weather,distance
<fct>,<chr>,<chr>,<chr>,<dbl>
Amy,trained,Time1,Rainy,89.15834
Amy,trained,Time1,Sunny,67.26723
Amy,trained,Time2,Rainy,78.03313
Amy,trained,Time2,Sunny,100.47912
Amy,trained,Time3,Rainy,67.87005
Amy,trained,Time3,Sunny,79.53061
Jack,trained,Time1,Rainy,64.27249
Jack,trained,Time1,Sunny,69.06735
Jack,trained,Time2,Rainy,90.20537


## Joining

The joining functions we'll be using come from a data library called `dplyr`.

You'll often have multiple data frames containing information about the same observations. In order to analyze all the information relating to the observations, you'll need to join them together into one data frame. There are many different types of joining methods depending on how you want to join (mostly with how to handle mis matches). This image goes over a subset of some of the more common types:

<img src='joins.png' height=40% width=40%>

Today we'll look only at `inner_join`, where groups are fully represented across two data frames. 

Imagine that, in addition to the data frame we have above about Jack, Ben, and Amy, we also have for each of these students in a different data frame their middle school grade level:

In [12]:
d_grade = data.frame(student = c('Amy', 'Jack', 'Ben'), grade = c('Sixth Grade', 'Eighth Grade', 'Eighth Grade'))
d_grade

student,grade
<chr>,<chr>
Amy,Sixth Grade
Jack,Eighth Grade
Ben,Eighth Grade


In order to analyze a question like whether throwing distance depends on grade level, for example, I'll need all of this data in one data frame. To do that, we use `inner_join`:

In [13]:
d = inner_join(long_data, d_grade, by = 'student')
d = arrange(d, student)
d

student,training_condition,time,weather,distance,grade
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
Amy,trained,Time1,Rainy,89.15834,Sixth Grade
Amy,trained,Time1,Sunny,67.26723,Sixth Grade
Amy,trained,Time2,Rainy,78.03313,Sixth Grade
Amy,trained,Time2,Sunny,100.47912,Sixth Grade
Amy,trained,Time3,Rainy,67.87005,Sixth Grade
Amy,trained,Time3,Sunny,79.53061,Sixth Grade
Ben,control,Time1,Rainy,92.98254,Eighth Grade
Ben,control,Time1,Sunny,77.79877,Eighth Grade
Ben,control,Time2,Rainy,69.65149,Eighth Grade


We can see that grade level automatically matches up relative to each student in the longer data frame.

# Assignment

For this assignment, you will perform many steps typically involved in a data analysis pipeline, culminating with summarizing, plotting, and reporting a result.

We'll be working with a dataset about grades from 10,000 students over the course of a year. The data are stored in two csv files in the same directory as this notebook, titled `wide_grades.csv` and `special_program.csv`. In each quarter of the year (1-4) each student was in both a basic materials class and an advanced materials class. The number of days absent and the average grade were recorded for each quarter for each class type. The format of the columns in `wide_grades.csv` is as follows:

`QuarterNumber_ClassType_Outcome`

Where `Outcome` is either days absent or average grade.

Some students were selected to be enrolled in a special program, where they were pulled out of class often to study at a more advanced level. Whether or not each student was enrolled in the special program is represented in the data from `special_program.csv`.

**Step 0:**  
Load in the following pacakges:

```
library(plyr)
library(dplyr)
library(tidyr)
library(ggplot2)
```

*If you get a big pink message saying, "You have loaded plyr after dplyr - this is likely to cause problems.", restart your kernel and then import these libraries. You might get other big pink messages, but these are usually fine.*

*Hint: You can get more info on the functions included in tidyr and dplyr [here](https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf).*

**Step 1:**  
Read in the data and store the data in two different data frames.  


**Step 2:**  
Look at the first few rows and summary statistics in the main data frame (the one with the grades and days absent).  

**Step 3:**  
Convert this wide dataset to long data. 

*Remember*: long data should have the following three properties  

1. Every column is a variable.  
2. Every row is an observation.  
3. Every cell is a single value.

*Hint 1*: You can do this using the following three functions: `gather`, `spread`, and `separate`.  
*Hint 2*: You can select multiple columns with the colon notation (eg, Column1:Column13).  
*Hint 3*: Try to imagine what the long data format will look like before you begin. What will each column and row represent?  
*Hint 4*: If you're struggling with Hint 3, I put a sample of what this long data might look like in `long_preview.png` in the same directory as this notebook.  


**Step 4:**  
We will want to analyze the trend between number of days absent and average grade. Because the students enrolled in the special program are missing class for systematic reasons that are different from why other students are missing class, we'll want to remove from the analysis all students enrolled in the special program.  

Join the data frame storing whether students are enrolled in the special program with the data frame storing average grades.  


**Step 5:**  
Drop all students who are enrolled in the special program from the data frame.  


**Step 6:**  
Use a custom summarization function and the `ddply` function to calculate the sum of days absent and the average grade for each student (see Lab 06). *Note: there are $10,000$ students in the dataset, so I wouldn't print the entire output of this summarization to the screen.*


**Step 7:**  
Use the `ggplot` syntax to make a scatter plot showing the relationship between number of days absent (x-axis) and grade (y-axis). 


**Step 8:**  
What can you infer about the relationship between days absent and average grades based on the plot?